# Тестирование модулей

## Модуль `datasets`

In [1]:
cd ..

C:\Users\kobel\учёба\recs\recs


In [2]:
import datasets

In [3]:
df_video_games = datasets.load_video_games()
df_video_games

,target
0,Wii Sports
1,Super Mario Bros.
2,Mario Kart Wii
3,Wii Sports Resort
4,Pokemon Red/Pokemon Blue
...,...
11557,15 Days
11558,Aiyoku no Eustia
11559,Woody Woodpecker in Crazy Castle 5
11560,LMA Manager 2007


## Модуль `models`

In [4]:
import models

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

wrapper_model = models.TfidfWrapperModel(tfidf)

In [6]:
array = wrapper_model.fit_transform(df_video_games.target)

print(array.shape)
array

(11562, 8838)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
# wrapper_model.save('testing')
# similarity = np.dot(emb, embedding[0]) / (np.linalg.norm(emb) * np.linalg.norm(embedding[0]) + 0.0001)

## `TEMP` code:

In [8]:
one_array = wrapper_model.transform(['mario'])

print(one_array.shape)
one_array

(1, 8838)


array([[0., 0., 0., ..., 0., 0., 0.]])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd


def get_similarity_df(array,
                      embedding,
                      k_best: int) -> pd.DataFrame:
    """
    Сравнение текущего эмбеддинга с базой данных и поиск k-подходящих адресов

    Данная функция берёт только один вариант для каждого идентификатора

    :param base_data: Словарь-справочник адресов
    :param embedding: Результаты преобразований
    :param k_best: Количество результатов, которые необходимо вывести
    :return: Фрейм данных с результатами работы модели
    """

    lst_target_id = []
    lst_value = []

    for target_id, emb in enumerate(array):
        similarity = cosine_similarity(embedding, [emb])[0][0]

        lst_target_id.append(target_id)
        lst_value.append(similarity)

    df_result = pd.DataFrame(
        {
            'target_id': lst_target_id,
            'value': lst_value,
        }
    )
    df_result = df_result.sort_values(by=['value'], ascending=False)
    df_result = df_result.head(k_best)

    return df_result

In [10]:
%%time
get_similarity_df(array, one_array, 10)

CPU times: total: 3.92 s
Wall time: 3.92 s


,target_id,value
479,479,0.723535
965,965,0.723535
538,538,0.723535
672,672,0.723535
750,750,0.723535
71,71,0.723535
910,910,0.723535
531,531,0.723535
381,381,0.723535
1055,1055,0.664143


In [11]:
df_video_games.iloc[479]

target    Mario Party
Name: 479, dtype: object

In [12]:
df_video_games.iloc[1055]

target    Mario Golf
Name: 1055, dtype: object

А если изменить одну букву?

In [13]:
one_array = wrapper_model.transform(['maeio'])

get_similarity_df(array, one_array, 10)

,target_id,value
0,0,0.0
7700,7700,0.0
7702,7702,0.0
7703,7703,0.0
7704,7704,0.0
7705,7705,0.0
7706,7706,0.0
7707,7707,0.0
7708,7708,0.0
7709,7709,0.0
